In [1]:
from pyspark.sql import SparkSession

In [2]:
spark = SparkSession.builder.appName("OTR").config("spark.sql.caseSensitive", "True").getOrCreate()

In [3]:
spark

In [47]:
from cs_go_analyse.Graphic_build.coord_managing import *
from cs_go_analyse.Graphic_build.heatmap import *
from cs_go_analyse.Graphic_build.vectorization import coordonee_bomb_site
from cs_go_analyse.Graphic_build.vectorization import distance_point
import json
import re
import numpy as np
import os
from pyspark.sql import SparkSession
import pyspark.sql.functions as F

map_wanted = "de_inferno"

def read_all_csgo_match_of_one_map_json(map_wanted):
    list_match=[]
    print("map_wanted :",map_wanted)
    for root, dirs, files in os.walk("demo_csgo/DataBase/"+map_wanted):
        for filename in files:
            pattern = "(.*?).json"
            filename = re.search(pattern, filename).group(1)
            print(filename)
            with open("demo_csgo/DataBase/"+map_wanted+"/"+filename+".json") as file:
                data=json.load(file)
                list_match.append(data)
    return list_match

def read_all_csgo_match_of_one_map_json_spark(map_wanted):

    # Get a list of all JSON files in the directory
    file_paths = [os.path.join(root, f) for root, dirs, files in os.walk("demo_csgo/DataBase/"+map_wanted) for f in files if f.endswith('.json')]

    dataframes = [spark.read.format("json").option("header", True).load(file_path) for file_path in file_paths]

    return dataframes

In [52]:

list_match = read_all_csgo_match_of_one_map_json_spark(map_wanted)

In [48]:
list_match =  read_all_csgo_match_of_one_map_json(map_wanted)

map_wanted : de_inferno
Memetits_de_inferno_1676583035_1-3a4ff066-b97f-482a-9d81-6014c9ffdc9e
Memetits_de_inferno_1676676678_1-355a99d9-90e3-47d9-a73b-2fecb28a15a0
Memetits_de_inferno_1677185051_1-a12489cb-6992-43c6-846f-6dbd82cebbb5
Memetits_de_inferno_1677188475_1-0ce98562-6e7d-435f-88f7-da61c8ec4a1d


In [49]:
list_match[0].keys()

dict_keys(['matchID', 'clientName', 'mapName', 'tickRate', 'playbackTicks', 'playbackFramesCount', 'parsedToFrameIdx', 'parserParameters', 'serverVars', 'matchPhases', 'matchmakingRanks', 'playerConnections', 'gameRounds'])

In [43]:
list_match[0].printSchema()

root
 |-- clientName: string (nullable = true)
 |-- gameRounds: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- bombEvents: array (nullable = true)
 |    |    |    |-- element: struct (containsNull = true)
 |    |    |    |    |-- bombAction: string (nullable = true)
 |    |    |    |    |-- bombSite: string (nullable = true)
 |    |    |    |    |-- clockTime: string (nullable = true)
 |    |    |    |    |-- playerName: string (nullable = true)
 |    |    |    |    |-- playerSteamID: long (nullable = true)
 |    |    |    |    |-- playerTeam: string (nullable = true)
 |    |    |    |    |-- playerX: double (nullable = true)
 |    |    |    |    |-- playerY: double (nullable = true)
 |    |    |    |    |-- playerZ: double (nullable = true)
 |    |    |    |    |-- seconds: double (nullable = true)
 |    |    |    |    |-- tick: long (nullable = true)
 |    |    |-- bombPlantTick: long (nullable = true)
 |    |    |-- ctBuyType: string (nullable

In [27]:
file_paths = spark.sparkContext.wholeTextFiles("demo_csgo/DataBase/"+"de_inferno"+"/*.json")

# Extract filenames and map to (filename, file_content) pairs
file_contents = file_paths.map(lambda file: (extract_filename(file[0]), file[1]))

# Load JSON data from file contents
json_data = file_contents.map(lambda x: (x[0], json.loads(x[1])))

json_data.printSchema()

AttributeError: 'PipelinedRDD' object has no attribute 'printSchema'